In [6]:
%cd ..

/Users/xiruihan/Desktop/RecFldGrn-main


# Load Data 

## Load Raw

In [11]:
# raw Data

import os 
import pandas as pd

# read the raw data here.
RecName = 'P'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)

# load all records
df = pd.read_csv(raw_file_path)
df

data/Simulations/P.csv


,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


## Select Columns

In [8]:
focal_ids = ['PID']
columns = ['age', 'basicInfo']

df = df[focal_ids + columns]
df

,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


## Basic Filtering

Cohort () & Time range ()

In [2]:
# nothing

## DF Rec Raw


with the name of `df_RecRaw`.

In [8]:
df_RecRaw = df

# Fields in Records

## Field: age

In [10]:
df['age']

0    52
1    21
2    23
3    77
4    23
5    73
6    34
7    76
Name: age, dtype: int64

## Field: basicInfo

In [12]:
df['basicInfo']

0      Male&B
1      Male&B
2    Female&B
3    Female&B
4      Male&A
5    Female&D
6    Female&A
7      Male&C
Name: basicInfo, dtype: object

## Call Parents

In [13]:
# nothing

## Pin Down

In [9]:
focal_ids = ['PID']
fields_cols = ['age', 'basicInfo']


df_proc = df[focal_ids + fields_cols]
df_proc

,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


# Save and Load

## Save

In [15]:
# from recfldgrn.datapoint import convert_PID_to_PIDgroup
# from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

def convert_PID_to_PIDgroup(x, RANGE_SIZE):
    if type(x) == str:
        if 'P' in x:
            int_value = int(x.replace('P', ''))
    else:
        int_value = int(x)
    range_size = RANGE_SIZE
    group_idx = int(int_value / range_size)
    s = group_idx * range_size
    e = (group_idx + 1) * range_size - 1
    groupname = f'Group_s{s}_e{e}'
    return groupname

def write_df_to_folders(DataName, data_folder, df, IDname):
    dfy = df.copy()
    # data_folder = f'../Data/B-EHRDataSet/{DataName}'
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    dfy['GroupName'] = dfy[IDname].apply(lambda x: convert_PID_to_PIDgroup(x , RANGE_SIZE))
    for groupname, dfx in dfy.groupby('GroupName'):
        file = os.path.join(data_folder, f'{groupname}.p')
        # print(file)
        dfx = dfx.drop(columns= ['GroupName'])
        dfx.to_pickle(file)
        
        
RANGE_SIZE = 10000

In [16]:
df_proc

,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


In [12]:
# the folder to save the Record.
###############
RecName = 'P'
###############





rec_folder = 'Data/ProcData/RecFolder'
IDname = 'PID' # always be PID, not review ID or other ID.

fullrec_folder = os.path.join(rec_folder, RecName)
print(fullrec_folder, '<----- fullrec_folder to save the records')
if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)

Data/ProcData/RecFolder/P <----- fullrec_folder to save the records


In [21]:
df = df_proc

write_df_to_folders(RecName, fullrec_folder, df, IDname)

## Load

In [22]:
def load_df_data_from_folder(data_folder, IDName):
    file_list = os.listdir(data_folder)
    # file_lis
    df = pd.concat([pd.read_pickle(os.path.join(data_folder, i)) for i in file_list if '.p' in i]).reset_index(drop = True)
    # df = df.sort_values(IDName).reset_index(drop = True)
    return df# .shape

In [23]:
IDName = 'PID'
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder, IDName)
print(df.shape)
df.head()

Data/ProcData/RecFolder/P
(8, 3)


,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A


## Load One Sample

In [25]:
def get_dfx_p_filename(IDValue, DataName, data_folder, RANGE_SIZE):
    group_name = convert_PID_to_PIDgroup(IDValue, RANGE_SIZE) 
    p_filename = f'{data_folder}/{DataName}/{group_name}.p'
    return p_filename
    
def load_patient_data(RID, IDValue, dfx):
    dfx = dfx[dfx[RID] == IDValue].reset_index(drop = True)
    return dfx

def get_dfx_from_buffer(RID, IDValue, DataName, data_folder, RANGE_SIZE, BUCKET_buffer):
    p_filename = get_dfx_p_filename(IDValue, DataName, data_folder, RANGE_SIZE)
    if p_filename not in BUCKET_buffer:
        dfx = pd.read_pickle(p_filename)
        BUCKET_buffer[p_filename] = dfx
        if len(BUCKET_buffer) > 100:
            for key in random.sample(BUCKET_buffer.keys(), 4):
                del BUCKET_buffer[key]
    else:
        dfx = BUCKET_buffer[p_filename]
    return dfx, BUCKET_buffer


class DataPoint(object):

    '''
    DataPoint: stands for one data point. 
    one instance is a patient.
    the instance of the patient could quickly the his/her record information by using `get_df_rec`.
    '''
    BUCKET_buffer = {}

    def __init__(self, RID, IDValue, data_folder, RANGE_SIZE):
        self.RID = RID
        self.IDValue = IDValue
        self.data_folder = data_folder
        self.df_rec_dict = {}
        self.RANGE_SIZE = RANGE_SIZE
        
    def get_df_rec(self, DataName):
        '''
        DataName: the name of the record, like: P, EC, BMI, PN, PNSect, etc.
        '''
        if DataName not in self.df_rec_dict:
            dfx, BUCKET_buffer = get_dfx_from_buffer(self.RID, self.IDValue, 
                                                     DataName, 
                                                     self.data_folder, 
                                                     self.RANGE_SIZE, 
                                                     self.BUCKET_buffer)
            self.BUCKET_buffer = BUCKET_buffer
            df_p = load_patient_data(self.RID, self.IDValue, dfx)
            self.df_rec_dict[DataName] = df_p
            return df_p
        else:
            return self.df_rec_dict[DataName]
        
    def __repr__(self):
        return f'<{self.RID} [{self.IDValue}]>'

In [29]:
# from recfldgrn.datapoint import PatientDP

# we want to study / explore P4 himself.

####################
RID = 'PID' # Record ID Name, always be PhyID
IDValue = 'P5'   # PhyID value
######################


data_folder = rec_folder
RANGE_SIZE = RANGE_SIZE

print(data_folder)


Pat = DataPoint(RID, IDValue, 
                data_folder, RANGE_SIZE)
Pat

Data/ProcData/RecFolder


<PID [P5]>

In [30]:
RecName

'P'

In [31]:
Pat.get_df_rec(RecName)

,PID,age,basicInfo
0,P5,73,Female&D
